In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss, make_scorer # type: ignore


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Read fingerprint file and experimental file

In [2]:
receptors = pd.read_csv(
    "/Users/xiaomuou620/Desktop/PRIVATE_DATA/hallucinome.csv"
) # selected only existing binding receptors left
fingerprints = pd.read_csv('../MACCS_fingerprints.csv')
print(receptors.head())



  receptor gprotein        drug    Emax  Emax_SE    TCoeff  TCoeff_SE  \
0    5HT1A      Gi1   25C-NBOMe  999.00   999.00       NaN        NaN   
1    5HT1A      Gi1    25I-NBMD   18.62     2.06  8.878197   0.201061   
2    5HT1A      Gi1   25I-NBOMe   24.53     3.28  7.947086   0.195392   
3    5HT1A      Gi1  25T7-NBOMe   22.00     3.87  7.457872   0.217788   
4    5HT1A      Gi1  4-AcO-MALT   38.93     5.16  7.417719   0.148907   

    logEmEC  logEmEc_Lower  logEmEc_Upper   pEC50  pEC50_SE  N  
0       NaN            NaN            NaN  999.00    999.00  3  
1  8.879980       8.529060       9.225551    7.61      0.30  3  
2  7.949698       7.607359       8.284201    6.56      0.28  3  
3  7.462423       7.078398       7.832796    6.12      0.30  3  
4  7.420284       7.158531       7.674340    5.83      0.20  3  


### Experimental data processing

In [3]:
# replace 999 in Emax with 0
receptors["Emax"] = receptors["Emax"].replace(999, 0)
# print(receptors.head())

# collect receptors for each drug
receptor_emax_mapping = (
    receptors.groupby(["drug", "receptor"])["Emax"].max().reset_index()
)
# print(receptor_emax_mapping.head())

# pivot the table to have drugs as rows and receptors as columns
receptor_emax_mapping_pivot = receptor_emax_mapping.pivot(
    index="drug", columns="receptor", values="Emax"
).reset_index()
print(receptor_emax_mapping_pivot.head())

receptor_emax_mapping_pivot.fillna(0, inplace=True)


receptor        drug      5HT1A       5HT1B   5HT1D   5HT1E       5HT1F  \
0          25C-NBOMe  13.047601    0.000000   61.56    0.00    0.000000   
1           25I-NBMD  27.200000   84.239297   91.94  105.26  102.099230   
2          25I-NBOMe  45.370000    0.000000   53.65   33.92   64.596780   
3         25T7-NBOMe  31.991315    0.000000   31.81    0.00   69.116817   
4         4-AcO-MALT  50.677696  126.170618  101.04  115.53  102.552716   

receptor   5HT2A  5HT2B   5HT2C  5HT5A  ...  Alpha2B  Alpha2C  Beta1AR  \
0         126.52  67.19   98.15   0.00  ...      0.0      0.0      0.0   
1         102.02  45.49   79.79   0.00  ...      0.0      0.0      0.0   
2         108.36  72.04  104.43  27.97  ...      0.0      0.0      0.0   
3          99.15  53.73   94.85  36.81  ...      0.0      0.0      0.0   
4         102.61  68.11  110.35  43.88  ...      0.0      0.0      0.0   

receptor  Beta2AR  Beta3AR  DRD1   DRD2   DRD3  DRD4   DRD5  
0             0.0      0.0   0.0  50.60   

### Fingerprint data processing

In [4]:
# Extract the names (first column)
drug_names_fingerprints = fingerprints['Name']
fingerprints_detail = fingerprints.drop('Name', axis=1)  # Drop the name column by index

# Create a dictionary where names are keys and fingerprints are lists of values
# csv_dict = {name: fingerprint.tolist() for name, fingerprint in zip(drug_names_fingerprints, fingerprints_detail.values)}


### Merge feature and labels

In [5]:
# merge receptor_emax_mapping_pivot with fingerprints
data_merged = pd.merge(
    receptor_emax_mapping_pivot, fingerprints, left_on="drug", right_on="Name"
).drop(columns=["Name"])

# data_merged.to_csv(
#     "/Users/xiaomuou620/Desktop/PRIVATE_DATA/selected_file_realval.csv", index=False
# )
print(data_merged.head())

         drug      5HT1A       5HT1B   5HT1D   5HT1E       5HT1F   5HT2A  \
0   25C-NBOMe  13.047601    0.000000   61.56    0.00    0.000000  126.52   
1    25I-NBMD  27.200000   84.239297   91.94  105.26  102.099230  102.02   
2   25I-NBOMe  45.370000    0.000000   53.65   33.92   64.596780  108.36   
3  25T7-NBOMe  31.991315    0.000000   31.81    0.00   69.116817   99.15   
4  4-AcO-MALT  50.677696  126.170618  101.04  115.53  102.552716  102.61   

   5HT2B   5HT2C  5HT5A  ...  157  158  159  160  161  162  163  164  165  166  
0  67.19   98.15   0.00  ...    1    1    1    1    1    1    1    1    1    0  
1  45.49   79.79   0.00  ...    1    1    1    1    1    1    1    1    1    0  
2  72.04  104.43  27.97  ...    1    1    1    1    1    1    1    1    1    0  
3  53.73   94.85  36.81  ...    1    1    1    1    1    1    1    1    1    0  
4  68.11  110.35  43.88  ...    1    1    1    1    1    1    1    1    1    0  

[5 rows x 193 columns]


## Training

In [6]:
# Drop the 'receptor' column
data_merged = data_merged.drop(columns=["drug"])


In [7]:
X = data_merged.iloc[:, 25:]
y = data_merged.iloc[:, 0:25]

print(X.shape, y.shape)
print(X.head())
print(y.head())


(40, 167) (40, 25)
   0  1  2  3  4  5  6  7  8  9  ...  157  158  159  160  161  162  163  164  \
0  0  0  0  0  0  0  0  0  0  0  ...    1    1    1    1    1    1    1    1   
1  0  0  0  0  0  0  0  0  0  0  ...    1    1    1    1    1    1    1    1   
2  0  0  0  0  0  0  0  0  0  0  ...    1    1    1    1    1    1    1    1   
3  0  0  0  0  0  0  0  0  0  0  ...    1    1    1    1    1    1    1    1   
4  0  0  0  0  0  0  0  0  0  0  ...    1    1    1    1    1    1    1    1   

   165  166  
0    1    0  
1    1    0  
2    1    0  
3    1    0  
4    1    0  

[5 rows x 167 columns]
       5HT1A       5HT1B   5HT1D   5HT1E       5HT1F   5HT2A  5HT2B   5HT2C  \
0  13.047601    0.000000   61.56    0.00    0.000000  126.52  67.19   98.15   
1  27.200000   84.239297   91.94  105.26  102.099230  102.02  45.49   79.79   
2  45.370000    0.000000   53.65   33.92   64.596780  108.36  72.04  104.43   
3  31.991315    0.000000   31.81    0.00   69.116817   99.15  53.73   94.85 

In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Regression

In [12]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

model_regression = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model_regression.fit(X_train, y_train)

# Predictions
y_pred = model_regression.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred, multioutput="raw_values")  # Per target
r2 = r2_score(y_test, y_pred, multioutput="variance_weighted")  # Overall R^2

print("MSE per receptor:", mse)
print("Weighted R^2:", r2)


MSE per receptor: [ 501.23408236  244.5208547   261.1945048   482.05662657  349.17163829
  117.99829144  364.45454302  255.09174132  964.02921457  507.41441547
 1064.07421372  308.63767725  585.43607379  349.47194551  866.87937352
  172.22913681 1133.88174511    0.           65.728202    443.04131615
  461.33188335  427.50991767  954.75841904 1297.0433401   940.10838405]
Weighted R^2: 0.3181223146944467


## Classification